In [2]:
import pandas as pd
import numpy as np
import urllib.request as ureq
from bs4 import BeautifulSoup
import re

In [ ]:
# =====|| For Implementation ||=====

# Define dictionaries to hold soup and path info

pathd_cat={}
soupd_cat={}
beg_path='https://www.rexelusa.com/'
cat_count=0
subcount=0
cat_names=[]
items=0

path='https://www.platt.com/platt-electric-supply/Lighting-Fixtures-and-Lighting-Controls/search.aspx?SectionID=11'
html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

linktable_cat=soup.find("div",{"class":"width100 Boxes"})
links_cat=linktable_cat.find_all('a')

df=pd.DataFrame() # define dataframe to append all dfsubs to

for link in links_cat:
       
    #print(link.get('href'))
    pathd_cat["path"+str(cat_count)]=link.get('href')
    tempath=beg_path+pathd_cat["path"+str(cat_count)]
    html=ureq.urlopen(tempath)


    soupd_cat["soup_cat"+str(cat_count)]=BeautifulSoup(html,'html.parser')
    type(soupd_cat["soup_cat"+str(cat_count)])
    title=soupd_cat["soup_cat"+str(cat_count)].title.get_text()
    nores=soupd_cat["soup_cat"+str(cat_count)].find("div",{"class":"no-results-found"})
    
    # check for no results pages
    if nores == None:
        print(">",title)

        pathd_sub={}
        soupd_sub={}
        subtitles=[]

        title_sub = soupd_cat["soup_cat"+str(cat_count)].title.get_text()
        if "Ballasts" not in title_sub:
            if "Drivers" not in title_sub:
                if "Outdoor Lighting" in title_sub: # test statement so i don't have to wait ages
                    linktable_sub=soupd_cat["soup_cat"+str(cat_count)].find("div",{"class":"width100 Boxes"})
                    
                    try:
                        links_sub=linktable_sub.find_all('a')

                        cat_count_sub=0
                        dfsub=pd.DataFrame() # define dataframe to append all dfsubsub's to

                        for link in links_sub:

                            pathd_sub["path"+str(subcount)]=link.get('href')
                            tempath=beg_path+pathd_sub["path"+str(subcount)]
                            html=ureq.urlopen(tempath)


                            soupd_sub["soup_cat"+str(subcount)]=BeautifulSoup(html,'html.parser')
                            type(soupd_sub["soup_cat"+str(subcount)])

                            subtitle=soupd_sub["soup_cat"+str(subcount)].title.get_text()

                            # check for no results pages
                            nores=soupd_sub["soup_cat"+str(subcount)].find("div",{"class":"no-results-found"})
                            if nores == None:
                                
                                dfsubsub=pd.DataFrame() # define dataframe to append all dfitems to

                                if "Lighting Poles" in subtitle:
                                    subtitles.append(subtitle)
                                    print(">>",subtitle)

                                    #subsubcategories
                                    pathd_subsub={}
                                    soupd_subsub={}
                                    subsubcount=0
                                    subsubtitles=[]

                                    # if there is no descendent, skip it
                                    linktable_subsub=soupd_sub["soup_cat"+str(subcount)].find("div",{"class":"width100 Boxes"})  

                                    try:
                                        links_subsub=linktable_subsub.find_all('a')
                                        cat_count_subsub=0

                                        #dfsubsub=pd.DataFrame() # define dataframe to append all dfitems to

                                        for link in links_subsub:

                                            #print(link.get('href'))
                                            pathd_subsub["path"+str(subsubcount)]=link.get('href')
                                            tempath=beg_path+pathd_subsub["path"+str(subsubcount)]
                                            html=ureq.urlopen(tempath)


                                            soupd_subsub["soup_cat"+str(subsubcount)]=BeautifulSoup(html,'html.parser')
                                            type(soupd_subsub["soup_cat"+str(subsubcount)])

                                            subsubtitle=soupd_subsub["soup_cat"+str(subsubcount)].title.get_text()

                                            # account for no descendents
                                            # look in item elements and look for products (if none there is one item)
                                            product_check=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"id":"productListContent"})
                                            if product_check==None:
                                                one_name=soupd_subsub["soup_cat"+str(subsubcount)].find_all("div",{"class":"last disabledLast"})[-1]
                                                subsub_onetitle=one_name.get_text('span')
                                                print(">>>",subsub_onetitle)
                                                print("There is one item within this sub sub category")
                                                # for now I will not gather information from the one category

                                                # ||||| ADD THIS ONE ITEM PAGE TO DATAFRAME ||||||

                                            else:

                                                # check for no results pages
                                                nores=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"no-results-found"})
                                                if nores == None:

                                                    subsubtitles.append(subsubtitle)
                                                    print(">>>",subsubtitle)

                                                    #==================================================================================
                                                    pathd_pagination={}
                                                    soupd_pagination={}
                                                    page_count=1

                                                    pathd_item={}
                                                    soupd_item={}
                                                    item_count=0

                                                    # grab last link in pagination, and create format for page url population
                                                    linktable_pagination=soupd_subsub["soup_cat"+str(subsubcount)].find("ul",{"class":"pagination OOTB"})
                                                    links_page=linktable_pagination.find_all('a')
                                                    lastlink=links_page[-2]

                                                    # account for pages where there may not be pagination
                                                    try:
                                                        lastpage=lastlink.get('href')
                                                        linkformat=beg_path+lastpage
                                                        last_pagenum=linkformat[-1]

                                                        # loop through the range of total pages and create urls, starting at 2 because we have already got the first page
                                                        pagination_urls=list()

                                                        for i in range(1,int(last_pagenum)+1):

                                                            # replace page values
                                                            url_new=linkformat[:int(-1)]+str(i)#+linkformat[int(-6)+1:]#               
                                                            pagination_urls.append(url_new)
                                                        lock=0

                                                    except AttributeError:
                                                        lock=1

                                                    # look at first item page
                                                    linktable_item=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"id":"productListContent"})
                                                    links_item=linktable_item.find_all('a',{"class":"productListCatNumLink"})
                                                    for link in links_item:
                                                        pathd_item["path"+str(item_count)]=link.get('href')

                                                        tempath=beg_path+pathd_item["path"+str(item_count)]
                                                        html=ureq.urlopen(tempath)

                                                        soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                                        type(soupd_item["soup_cat"+str(item_count)])
                                                        itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                                        #print(testtitle)
                                                        item_count+=1

                                                    print("There are",item_count,"items on page 1")
                                                    page1_cnt=item_count

                                                    pgitem_count=0    


                                                    # loop through remaining pages if there are any

                                                    if lock==0:
                                                        for page in pagination_urls:

                                                            # get soup for that page
                                                            html=ureq.urlopen(page)
                                                            soup=BeautifulSoup(html,'html.parser')
                                                            type(soup)

                                                            # grab item page links on pagination page
                                                            linktable_item=soup.find("div",{"class":"SearchProductsTable"})
                                                            links=linktable_item.find_all('a',{"class":"productListCatNumLink"})

                                                            pgitem_count=0
                                                            for link in links:
                                                                pathd_item["path"+str(item_count)]=link.get('href')

                                                                tempath=beg_path+pathd_item["path"+str(item_count)]
                                                                html=ureq.urlopen(tempath)

                                                                soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                                                type(soupd_item["soup_cat"+str(item_count)])
                                                                itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                                                #print(itemtitle)
                                                                item_count+=1
                                                                pgitem_count+=1
                                                            page_count+=1

                                                            print("page:",page_count,", items:",pgitem_count,)
                                                    print("There are",page_count,"page(s) in this descendent")
                                                    print("There are",item_count,"total items")
                                                    if lock==1:
                                                        print("There is only one page")

                                                    # ===== DATAFRAME INFORMATION =====
                                                    # With soups for all item pages, iterate through soups to gather information, then create dataframe from them.

                                                    # define dataframe to hold item page info
                                                    dfitem=pd.DataFrame() #********ISSUE: DF REDEFINED EVERY LOOP
                                                    pg_count=0

                                                    # loop through item pages
                                                    for item in soupd_item:
                                                        itemtitle=soupd_item['soup_cat'+str(pg_count)].title.get_text()
                                                        #print(itemtitle)

                                                        # find price box, strips away the tag info and leaves the price. If price isn't listed, skip
                                                        try:
                                                            price=soupd_item['soup_cat'+str(pg_count)].find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)
                                                        except AttributeError:
                                                            price='NaN'

                                                        # find description for product
                                                        description=soupd_item[item].find("span",{"id":"lblDetailDes"}).get_text(strip=True)

                                                        prodinfo=[]
                                                        prodinfo.append(itemtitle)
                                                        prodinfo.append(description)
                                                        prodinfo.append(price)
                                                        df0=pd.DataFrame(prodinfo)
                                                        df0=df0.transpose()
                                                        df0.columns=['title','description','price']
                                                        df0.index=['Value']


                                                        # find specification table
                                                        spectable=soupd_item[item].find("table",{"class":"seoSpecTable1"})

                                                        # loop through spec table and grab all attributes
                                                        specrows=spectable.find_all('tr')

                                                        # improved method to grab columns within each row
                                                        list_rows=[]
                                                        for row in specrows:
                                                            cells=row.find_all('td')
                                                            str_cells=str(cells)
                                                            clean=re.compile('<.*?>')
                                                            clean2=(re.sub(clean,'',str_cells))
                                                            list_rows.append(clean2)
                                                        type(clean2)

                                                        # pass array to a dataframe
                                                        df1=pd.DataFrame(list_rows)

                                                        # use split function to separate headers from values
                                                        df1_1=df1[0].str.split(',',n=1,expand=True)
                                                        df1_1.columns=['Name','Value']

                                                        # Clean up data
                                                        df1_1['Name']=df1_1['Name'].str.replace('[','')
                                                        df1_1['Name']=df1_1['Name'].str.replace(':','')
                                                        df1_1['Value']=df1_1['Value'].str.replace(']','')

                                                        # transpose dataframe because the headers are on the rows
                                                        df1_1=df1_1.transpose()

                                                        # replace headers with first row and delete first row
                                                        df1_2=df1_1.rename(columns=df1_1.iloc[0])
                                                        df1_3=df1_2.drop(df1_2.index[0])

                                                        # find second specification table
                                                        spectable2=soupd_item[item].find("table",{"class":"seoSpecTable2"})

                                                        # loop through spec table and grab all attributes
                                                        specrows2=spectable2.find_all('tr')

                                                        # improved method to grab columns within each row
                                                        list_rows2=[]
                                                        for row in specrows2:
                                                            cells=row.find_all('td')
                                                            str_cells=str(cells)
                                                            clean=re.compile('<.*?>')
                                                            clean2=(re.sub(clean,'',str_cells))
                                                            list_rows2.append(clean2)
                                                        type(clean2)

                                                        # pass array to a dataframe
                                                        df2=pd.DataFrame(list_rows2)

                                                        # use split function to separate headers from values
                                                        df2_1=df2[0].str.split(',',n=1,expand=True)
                                                        df2_1.columns=['Name','Value']

                                                        # Clean up data
                                                        df2_1['Name']=df2_1['Name'].str.replace('[','')
                                                        df2_1['Name']=df2_1['Name'].str.replace(':','')
                                                        df2_1['Value']=df2_1['Value'].str.replace(']','')

                                                        # transpose dataframe because the headers are on the rows
                                                        df2_1=df2_1.transpose()

                                                        # replace headers with first row and delete first row
                                                        df2_2=df2_1.rename(columns=df2_1.iloc[0])
                                                        df2_3=df2_2.drop(df2_2.index[0])

                                                        # merge this dataframe with the other
                                                        dfmer=df1_3.join(df2_3,sort=True)
                                                        dfmer=df0.join(dfmer,sort=True)
                                                        dfmer['subsubcategory']=subsubtitle
                                                        dfitem=dfitem.append(dfmer,sort=True)
                                                        pg_count+=1
                                                        items+=1
                                                    #==================================================================================

                                                    # append each subsubcategory data
                                                    dfsubsub['subcategory']=subtitle
                                                    dfsubsub=dfsubsub.append(dfitem,sort=True)
                                                    cat_count_subsub+=1
                                                    subsubcount+=1
                                                else:
                                                    cat_count_subsub+=1
                                                    print("no results found")

                                        print("There are",cat_count_subsub,"sub sub categories within this subcategory")  
                                                    # So every other entry in the dictionary is the html page

                                    except AttributeError:
                                        print("There are no more descendents in this subcategory")

                                # append each subcategory to dataframe
                                dfsub['category']=title 
                                dfsub=dfsub.append(dfsubsub,sort=True)

                                cat_count_sub+=1
                                subcount+=1
                            else: 
                                cat_count_sub+=1
                                print("no results found")

                        print("There are",cat_count_sub,"sub categories within this category")  

                        # append each category dataframe
                        #df['group']=title  
                        df=df.append(dfsub,sort=True)
                    except AttributeError:
                        cat_count+=1
                        print("This category seems to not have any descendents")

            else:
                cat_count+=1
                print("This category contains only drivers and will be skipped")
        else: 
            cat_count+=1
            print("This category contains only ballasts and will be skipped")

        # append each category dataframe
        #df['group']=title  
        #df=df.append(dfsub,sort=True)

        cat_count+=1  
        print("subcategory successfully scraped")
    else:
        cat_count+=1
        print("reuslts not found")

#reindex
df.index=np.arange(1,df.title.count()+1,1) 
    
print("There are",cat_count,"categories")  
print("There are",items,"total items")
print("scraping success!")

df.to_csv("Exports/platt_lighting_pole.csv")

> 
	Dimmers/Dimming Controls - Platt Electric Supply

subcategory successfully scraped
> 
	Emergency Lighting & Signs - Platt Electric Supply

subcategory successfully scraped
> 
	Explosion Proof/Hazardous/Processing - Platt Electric Supply

subcategory successfully scraped
> 
	Lamps, Bulbs, Ballasts - Platt Electric Supply

This category contains only ballasts and will be skipped
subcategory successfully scraped
> 
	Indoor Lighting - Platt Electric Supply

subcategory successfully scraped
> 
	Indoor Lighting - Recessed - Platt Electric Supply

subcategory successfully scraped
> 
	Lampholders, Adaptors & Extensions - Platt Electric Supply

subcategory successfully scraped
> 
	Lighting Control Systems - Platt Electric Supply

subcategory successfully scraped
> 
	Occupancy Sensors - Platt Electric Supply

subcategory successfully scraped
> 
	Outdoor Lighting - Platt Electric Supply

>> 
	Lighting Poles - Platt Electric Supply

>>> 
	LED Pole Fixtures - Platt Electric Supply

There are 13

In [12]:
# pagination is set up a bit differently, here the second to last link is the last one

path='https://www.rexelusa.com/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041'
html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

linktable_pagination=soup.find("ul",{"class":"pagination OOTB"})
links_page=linktable_pagination.find_all('a')
#lastlink=linktable_pagination.find('a',{"class":"pg-arrow-last"})
lastlink=links_page[-2]

<a href="/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&amp;isOnlyProductIds=&amp;isOrderForm=&amp;skuIndex=&amp;isCreateOrderForm=&amp;searchResultType=&amp;pageSize=12&amp;filter=&amp;aliasname=&amp;viewMode=gridView&amp;sort=&amp;isPaginationRqst=true&amp;text=&amp;page=3">4</a>

In [21]:
# account for pages where there may not be pagination
beg_path='https://www.rexelusa.com/'

lastpage=lastlink.get('href')
linkformat=beg_path+lastpage
last_pagenum=linkformat[-1]

# loop through the range of total pages and create urls, starting at 2 because we have already got the first page
pagination_urls=list()
#print(last_pagenum)

for i in range(1,int(last_pagenum)+1):

    # replace page values
    url_new=linkformat[:int(-1)]+str(i)#+linkformat[int(-1)+1:] removed because integer is at very end
    pagination_urls.append(url_new)
pagination_urls

['https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=1',
 'https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=2',
 'https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=3']

In [16]:
pagination_urls

['/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=2/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=3',
 '/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=3/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&

In [82]:
# pagination is set up a bit differently, here the second to last link is the last one

path='https://www.rexelusa.com/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=1'
html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

linktable=soup.find("div",{"id":"productListContent"})
links=linktable.find_all('a')
for link in links:
    print(beg_path+str(link.get('href')))

https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=0
https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=0
https://www.rexelusa.com//usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/c/CT4041?q=%3Arelevance&isOnlyProductIds=&isOrderForm=&skuIndex=&isCreateOrderForm=&searchResultType=&pageSize=12&filter=&aliasname=&viewMode=gridView&sort=&isPaginationRqst=true&text=&page=2
https://www.rexelusa.com//usr/Root-Category/Lighting

In [77]:
ab=linktable.find('h4',{"class":"grid-product-name"})
arr=[]
for stuff in ab.children:
    arr.append(stuff)
    print(stuff)
letters=list(ab.contents)
letters[0]



<a baynote_bnrank="" baynote_irrank="" href="/usr/Root-Category/Lighting%2C-Lighting-Controls/Outdoor-Lighting/Lighting-Poles/HID-Pole-Fixtures/Lithonia-Lighting-115-07S-RN-120-R3-FG-Roadway-Lighting%2C-70W%2C-HPS%2C-120V/p/611801?prevPageNumber=1" title="Lithonia Lighting 115-07S-RN-120-R3-FG Roadway Lighting, 70W, HPS, 120V">Lithonia Lighting 115-07S-RN-120-R3-FG Roadway Lighting, 70W, HPS, 120V</a>




'\n'